In [2]:
import tkinter as tk
from tkinter import messagebox
import threading
import cv2
import numpy as np
import os
import mediapipe as mp

# Import functions from nn.py and collectdata.py
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, lh, rh])

def load_model():
    # Load your trained model here
    model_path = 'action.h5'  # Update with the correct path to your model file
    # Placeholder function to load model; replace with your actual implementation
    print(f"Loading model from {model_path}")
    # Example usage: model = your_load_model_function(model_path)

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # Your implementation of drawing styled landmarks
    pass

# Global variables
model = None

def load_trained_model():
    global model
    try:
        mp_holistic = mp.solutions.holistic
        mp_drawing = mp.solutions.drawing_utils
        model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    except Exception as e:
        messagebox.showerror("Error", f"Error loading Mediapipe model: {e}")

def start_real_time_detection():
    messagebox.showinfo("Real-Time Detection", "Starting Real-Time Detection...")
    if model is None:
        messagebox.showerror("Error", "Mediapipe model not loaded. Please load the model first.")
        return
    
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Make detections
        image, results = mediapipe_detection(frame, model)

        # Process keypoints
        keypoints = extract_keypoints(results)

        # Draw styled landmarks
        draw_styled_landmarks(image, results)

        # Display the image
        cv2.imshow('Real-Time Detection', image)

        # Check for 'q' key press to exit
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Create the main Tkinter window
root = tk.Tk()
root.title("Sign Language Recognition App")

# Load Model button
load_model_button = tk.Button(root, text="Load Model", command=load_trained_model)
load_model_button.pack(pady=100)

# Start Real-Time Detection button
start_detection_button = tk.Button(root, text="Start Real-Time Detection", command=start_real_time_detection)
start_detection_button.pack(pady=100)

# Run the main event loop
root.mainloop()
